<a href="https://colab.research.google.com/github/Ravimk07/Plant-disease-classification/blob/master/Potato_image_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
os.getcwd()
os.listdir(os.getcwd())
os.chdir('/content')
os.getcwd()

'/content'

In [19]:
from google.colab import files
uploaded = files.upload()

Saving Potato_crop.zip to Potato_crop.zip


In [0]:
# This is for extracting the files
import zipfile
path_to_zip_file = '/content/Potato_crop.zip'
directory_to_extract_to = '/content/'
zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
zip_ref.extractall(directory_to_extract_to)
zip_ref.close()

In [22]:
for name, data in uploaded.items():
  with open(name, 'wb') as f:
    f.write(data)
    print('Saved file', name)

Saved file Potato_crop.zip


In [0]:
# CNN MODEL
import os
import glob
import ntpath
import cv2
import numpy as np
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalMaxPooling2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

In [0]:
PREPROCESSED_DATA_FOLDER= "/content/Potato_crop"

In [0]:
images = []
classes = []

imgNone = 0
imgCnt = 0

for classFolderPath in os.listdir(PREPROCESSED_DATA_FOLDER):
    
    pattern = os.path.join(PREPROCESSED_DATA_FOLDER, classFolderPath, "*.JPG")
    
    for imagePath in glob.glob( pattern ):

        img = cv2.imread(imagePath, cv2.IMREAD_COLOR)
        
        # if the file contain any missig value ignore it
        if img is None:
            imgNone += 1
            continue
            
        imgCnt += 1
        
        images.append( img )
        classes.append(classFolderPath)

In [0]:
def resize_images(img):

  img = np.array(img).astype(np.uint8)
  #print(img.dtype)
  res = cv2.resize(img,(224,224), interpolation = cv2.INTER_CUBIC)
  return res
#resized=[]
#for image in images:
#  ressized= resize_images(images[image])

In [0]:
#save resized images into images.
images = [resize_images(img) for img in images]

In [0]:
images = np.array(images)
classes = np.array(classes)

In [0]:

import sys 
import os
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as k
import keras
from keras.callbacks import TensorBoard
from keras.applications.mobilenet import MobileNet

# fix seed for reproducible results (only works on CPU, not GPU)
seed = 9
np.random.seed(seed=seed)
tf.set_random_seed(seed=seed)

In [0]:

# All defined hyper parameters for model
nb_classes = 2                             # number of classes
based_model_last_block_layer_number = 126  # value is based on based model selected.
img_width, img_height = 256, 256           # change based on the shape/structure of your images for squeezenet
batch_size = 32                            # dependent on CPU/GPU memory capacity (powers of 2 values).
nb_epoch = 20                              # number of iteration the algorithm gets trained.
learn_rate = 1e-4                          # sgd learning rate
momentum = .9                              # sgd momentum to avoid local minimum
transformation_ratio = .05                 # how aggressive will be the data augmentation/transformation
nb_train_samples = 240                     # Total number of train samples. NOT including augmented images
nb_validation_samples = 96                 # Total number of train samples. NOT including augmented images.

In [0]:
images = np.array(images)
classes = np.array(classes)
# Label encoder
from sklearn.preprocessing import LabelBinarizer
enc = LabelBinarizer()
encodedLabels = enc.fit_transform(classes)

In [0]:
# Spliting data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, encodedLabels, test_size=0.2, random_state=80)

In [0]:
from keras.preprocessing import image
from keras.applications import resnet50, inception_v3, vgg16
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.optimizers import Adam
import numpy as np

batch_size = 50
num_classes = 3

#base_model = resnet50.ResNet50
#base_model = inception_v3.InceptionV3
base_model = vgg16.VGG16

In [41]:
base_model = base_model(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(lr=0.0001),
              metrics=['acc'])

x_train = np.random.normal(loc=127, scale=127, size=(50, 224,224,3))
y_train = np.array([0,1]*25)
x_train = vgg16.preprocess_input(x_train)

58892288/58889256 [==============================] - 5s 0us/step


In [42]:
print(model.evaluate(x_train, y_train, batch_size=batch_size, verbose=0))
model.fit(x_train, y_train,
          epochs=100,
          batch_size=batch_size,
          shuffle=False,
          validation_data=(x_train, y_train))

[4.3350749015808105, 0.5]
Instructions for updating:
Use tf.cast instead.
Train on 50 samples, validate on 50 samples
Epoch 1/100
50/50 [==============================] - 2s 38ms/step - loss: 4.3351 - acc: 0.5000 - val_loss: 3.2564 - val_acc: 0.5000
Epoch 2/100
50/50 [==============================] - 1s 22ms/step - loss: 3.2564 - acc: 0.5000 - val_loss: 2.2170 - val_acc: 0.5000
Epoch 3/100
50/50 [==============================] - 1s 22ms/step - loss: 2.2170 - acc: 0.5000 - val_loss: 1.3162 - val_acc: 0.5000
Epoch 4/100
50/50 [==============================] - 1s 22ms/step - loss: 1.3162 - acc: 0.5000 - val_loss: 0.9147 - val_acc: 0.4000
Epoch 5/100
50/50 [==============================] - 1s 22ms/step - loss: 0.9147 - acc: 0.4000 - val_loss: 1.1952 - val_acc: 0.5000
Epoch 6/100
50/50 [==============================] - 1s 22ms/step - loss: 1.1952 - acc: 0.5000 - val_loss: 1.5304 - val_acc: 0.5000
Epoch 7/100
50/50 [==============================] - 1s 22ms/step - loss: 1.5304 - acc: 0.